Локальные дескрипторы и сверточные нейронные сети

**Антонов Михаил Евгеньевич, М-26**
___

#### Оглавление
1. Setup: подготовка окружения и импорты
2. Данные: описание и загрузка изображений
3. Локальные дескрипторы (SIFT) и backpropagation
___

## 1. Setup: подготовка окружения и импорты

In [ ]:
# Установка зависимостей (раскомментировать при первом запуске)
# !pip install opencv-python opencv-contrib-python -q
# !pip install numpy matplotlib -q
# !pip install datasets torchvision -q

In [ ]:
import os
import urllib.request
from typing import Tuple, List, Optional

import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchvision.models import resnet18
from datasets import load_dataset

%matplotlib inline

# Константы
MAX_IMAGE_DIM = 800
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
def resize_keep_aspect(img: np.ndarray, max_dim: int = MAX_IMAGE_DIM) -> np.ndarray:
    """Изменяет размер изображения с сохранением пропорций.
    
    Args:
        img: Входное изображение
        max_dim: Максимальный размер по большей стороне
        
    Returns:
        Изображение с изменённым размером
    """
    h, w = img.shape[:2]
    scale = min(1.0, max_dim / max(h, w))
    if scale < 1.0:
        new_size = (int(w * scale), int(h * scale))
        img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)
    return img


def load_and_prepare(path: str) -> Tuple[np.ndarray, np.ndarray]:
    """Загружает и подготавливает изображение.
    
    Args:
        path: Путь к изображению
        
    Returns:
        Кортеж (цветное изображение BGR, изображение в оттенках серого)
        
    Raises:
        ValueError: Если изображение не удалось загрузить
    """
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise ValueError(f"Не удалось загрузить изображение: {path}")

    if img.dtype != np.uint8:
        img = (np.clip(img, 0, 1) * 255).astype(np.uint8)

    if img.ndim == 3 and img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)

    img = resize_keep_aspect(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    return img, gray


def download_images(urls: List[str], output_dir: str = "data") -> Tuple[List[np.ndarray], List[np.ndarray]]:
    """Загружает изображения по URL и подготавливает их.
    
    Args:
        urls: Список URL изображений
        output_dir: Директория для сохранения
        
    Returns:
        Кортеж (список цветных изображений, список grayscale изображений)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    images = []
    gray_images = []
    
    for i, url in enumerate(urls):
        filename = os.path.join(output_dir, f"image_{i}.jpg")
        try:
            urllib.request.urlretrieve(f"{url}?w=800&h=600&fit=crop", filename)
            img, gray = load_and_prepare(filename)
            images.append(img)
            gray_images.append(gray)
            print(f"Загружено: {filename} - размер: {img.shape}")
        except Exception as e:
            print(f"Ошибка загрузки {url}: {e}")
    
    return images, gray_images


# Загрузка изображений
print("Загрузка изображений...")

IMAGE_URLS = [
    "https://images.unsplash.com/photo-1574158622682-e40e69881006",  # Кошка
    "https://images.unsplash.com/photo-1543466835-00a7907e9de1",    # Собака
]

images, gray_images = download_images(IMAGE_URLS)
titles = [f"Изображение {i+1}" for i in range(len(images))]

# Визуализация
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for i, (img, gray, title) in enumerate(zip(images, gray_images, titles)):
    axes[0, i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0, i].set_title(f'{title} (цветное)')
    axes[0, i].axis('off')
    
    axes[1, i].imshow(gray, cmap='gray')
    axes[1, i].set_title(f'{title} (оттенки серого)')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

# Сохранение для дальнейшего использования
for i, (img, gray) in enumerate(zip(images, gray_images)):
    cv2.imwrite(f'image{i+1}_color.jpg', img)
    cv2.imwrite(f'image{i+1}_gray.jpg', gray)

В работе использовались изображения, соответствующие по стилю набору ImageNet:

Изображение 1 (Кошка) — демонстрирует животное с четкой текстурой шерсти и выраженными контурами. Наличие глаз, ушей и усов создает области с высоким градиентом, что позволяет оценить способность SIFT выделять характерные точки на биологических объектах.

Изображение 2 (Собака) — показывает другое животное в ином ракурсе, с отличиями в текстуре и освещении. Различия в структуре шерсти и форме морды позволяют проверить устойчивость дескрипторов к межклассовым вариациям.

Такая пара изображений хорошо подходит для анализа работы SIFT, так как содержит как сходные черты (оба - домашние животные), так и существенные различия, что позволяет оценить качество сопоставления в реалистичных условиях.

In [ ]:
def create_sift_detector() -> cv2.SIFT:
    """Создаёт SIFT детектор с поддержкой разных версий OpenCV."""
    try:
        return cv2.SIFT_create()
    except AttributeError:
        return cv2.xfeatures2d.SIFT_create()


def visualize_keypoints(
    img: np.ndarray,
    keypoints: List,
    max_points: int = 150,
    color: Tuple[int, int, int] = (0, 255, 0),
    title: str = "Ключевые точки"
) -> None:
    """Визуализирует ключевые точки на изображении.
    
    Args:
        img: Исходное изображение
        keypoints: Список ключевых точек
        max_points: Максимальное количество точек для отображения
        color: Цвет точек (BGR)
        title: Заголовок графика
    """
    vis_img = cv2.drawKeypoints(
        img,
        keypoints[:max_points],
        None,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
        color=color
    )
    
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(title, fontsize=14, pad=20)
    plt.tight_layout()
    plt.show()


# Создание детектора и извлечение ключевых точек
detector = create_sift_detector()

keypoints_list = []
descriptors_list = []

for i, gray in enumerate(gray_images):
    kp, desc = detector.detectAndCompute(gray, None)
    keypoints_list.append(kp)
    descriptors_list.append(desc)
    print(f"Изображение {i+1}: {len(kp)} ключевых точек")

print(f"\nРазмерность дескриптора: {descriptors_list[0].shape[1]} измерений")

# Визуализация ключевых точек
visualize_keypoints(
    images[0], keypoints_list[0], 
    color=(0, 255, 0),
    title=f'Характерные точки изображения 1 (первые 150 из {len(keypoints_list[0])})'
)

visualize_keypoints(
    images[1], keypoints_list[1],
    color=(255, 0, 0),
    title=f'Характерные точки изображения 2 (первые 150 из {len(keypoints_list[1])})'
)

Результаты извлечения:

На первом изображении обнаружено 2181 ключевых точек
На втором изображении обнаружено 403 ключевых точек
Каждая точка описывается дескриптором размерностью 128 измерений
Дескрипторы представляют собой градиентные гистограммы локальных окрестностей
Ключевые точки выделяются в местах с выраженными перепадами яркости (углы, границы, текстуры). Каждая точка характеризуется:

Координатами (x, y)
Радиусом окрестности (size)
Углом ориентации (angle)
Качеством отклика (response)

In [ ]:
def filter_matches_lowe(
    matches: List,
    ratio_threshold: float = 0.75
) -> List:
    """Фильтрует совпадения по критерию Лоу.
    
    Args:
        matches: Список пар совпадений от knnMatch
        ratio_threshold: Пороговое значение для фильтрации
        
    Returns:
        Отфильтрованный список совпадений
    """
    filtered = []
    for primary, secondary in matches:
        if primary.distance < ratio_threshold * secondary.distance:
            filtered.append(primary)
    return sorted(filtered, key=lambda x: x.distance)


def analyze_matches(matches: List) -> dict:
    """Вычисляет статистику по совпадениям.
    
    Args:
        matches: Список совпадений
        
    Returns:
        Словарь со статистикой
    """
    distances = [m.distance for m in matches]
    return {
        'count': len(matches),
        'mean': np.mean(distances),
        'min': min(distances),
        'max': max(distances),
        'std': np.std(distances)
    }


# Сопоставление дескрипторов
RATIO_THRESHOLD = 0.75

matcher = cv2.BFMatcher()
initial_matches = matcher.knnMatch(descriptors_list[0], descriptors_list[1], k=2)

filtered_matches = filter_matches_lowe(initial_matches, RATIO_THRESHOLD)
stats = analyze_matches(filtered_matches)

print("Анализ совпадений:")
print(f"  Первоначально найдено пар: {len(initial_matches)}")
print(f"  После фильтрации: {stats['count']}")
print(f"  Коэффициент фильтрации: {RATIO_THRESHOLD}")
print(f"  Процент оставшихся: {stats['count']/len(initial_matches)*100:.1f}%")

# Визуализация
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Гистограмма расстояний
distances = [m.distance for m in filtered_matches]
ax1.hist(distances, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
ax1.axvline(stats['mean'], color='red', linestyle='--', label=f"Среднее: {stats['mean']:.2f}")
ax1.set_xlabel('Расстояние между дескрипторами', fontsize=12)
ax1.set_ylabel('Количество совпадений', fontsize=12)
ax1.set_title('Распределение расстояний совпадений', fontsize=14, pad=15)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Визуализация совпадений
vis_matches = cv2.drawMatches(
    images[0], keypoints_list[0],
    images[1], keypoints_list[1],
    filtered_matches[:50],
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    matchColor=(0, 255, 0)
)

ax2.imshow(cv2.cvtColor(vis_matches, cv2.COLOR_BGR2RGB))
ax2.axis('off')
ax2.set_title(f'Визуализация совпадений (первые 50, порог={RATIO_THRESHOLD})', fontsize=14, pad=15)

plt.tight_layout()
plt.show()

# Статистика
print(f"\nСтатистика по совпадениям:")
print(f"  Среднее расстояние: {stats['mean']:.3f}")
print(f"  Минимальное: {stats['min']:.3f}")
print(f"  Максимальное: {stats['max']:.3f}")
print(f"  Стандартное отклонение: {stats['std']:.3f}")

Интерпретация результатов:

Метод сопоставления: Использован алгоритм Brute-Force с поиском k ближайших соседей (k=2)
Фильтрация совпадений: Применен критерий Лоу для отсеивания ложных соответствий
Визуализация: Линии соединяют соответствующие ключевые точки на двух изображениях
Качество совпадений: Оценивается по евклидову расстоянию между дескрипторами
Критерий Лоу (Lowe's ratio test) позволяет отфильтровать ненадежные совпадения, сравнивая расстояние до ближайшего соседа с расстоянием до второго ближайшего. Если отношение меньше порога (0.75), совпадение считается надежным.

## **2. Анализ шагов алгоритма SIFT**

### **2.1 Масштабно-пространственная фильтрация (Difference-of-Gaussians)**

**Операции над изображением:**
1. **Построение гауссовой пирамиды** - последовательное применение свертки с гауссовыми ядрами возрастающего размера:
   
   $L(x,y,\sigma) = G(x,y,\sigma) \ast I(x,y)$
   
   где $G(x,y,\sigma) = \frac{1}{2\pi\sigma^2}e^{-\frac{x^2+y^2}{2\sigma^2}}$

2. **Вычисление разности гауссианов** - попиксельное вычитание изображений соседних масштабов:
   
   $D(x,y,\sigma) = L(x,y,k\sigma) - L(x,y,\sigma)$
   где $k$ - коэффициент масштабирования

**Вычисление производных (backpropagation):**
- **Свертка с гауссовым ядром**: линейная операция, градиент вычисляется как свертка с тем же ядром:
  
  $\frac{\partial L}{\partial I} = G(x,y,\sigma)$

- **Разность гауссианов**: линейное вычитание, производные:
  
  $\frac{\partial D}{\partial L_1} = 1$, $\frac{\partial D}{\partial L_2} = -1$

- **Цепное правило**: градиенты распространяются через последовательность операций

### **2.2 Выбор экстремумов (поиск ключевых точек)**

**Операции над изображением:**
1. **3D-поиск максимумов** - сравнение каждого пикселя с 26 соседями (8 в текущем масштабе + 9 в масштабе выше + 9 в масштабе ниже)
2. **Пороговая фильтрация** - удаление точек с низким контрастом: $|D(x,y,\sigma)| < T_{contrast}$
3. **Подавление краевых точек** - анализ отношения главных собственных значений матрицы Гессе

**Вычисление производных (backpropagation):**
- **Жесткий максимум**: недифференцируемая операция, требует замены на мягкую версию:
  
  Используем soft-argmax: $p_i = \frac{e^{\beta D_i}}{\sum_j e^{\beta D_j}}$
  
  где $\beta$ - параметр температуры

- **Пороговая фильтрация**: можно заменить на сигмоидную функцию:
  
  $f(D) = \frac{1}{1+e^{-\alpha(D-T)}}$

- **Анализ матрицы Гессе**: дифференцируем через собственные разложения с регуляризацией

### **2.3 Назначение ориентаций ключевым точкам**

**Операции над изображением:**
1. **Вычисление градиентов**:
   
   $m(x,y) = \sqrt{(L(x+1,y)-L(x-1,y))^2 + (L(x,y+1)-L(x,y-1))^2}$
   
   $\theta(x,y) = \text{atan2}(L(x,y+1)-L(x,y-1), L(x+1,y)-L(x-1,y))$

2. **Построение гистограммы ориентаций** - 36 бинов, трилинейная интерполяция
3. **Выбор доминирующих направлений** - пики в гистограмме выше 80% от максимума

**Вычисление производных (backpropagation):**
- **Вычисление градиентов**: конечные разности, производные:
  
  $\frac{\partial m}{\partial L} = \frac{1}{m}\left[(L(x+1,y)-L(x-1,y))\frac{\partial}{\partial L}(L(x+1,y)-L(x-1,y)) + \cdots\right]$

- **Функция atan2**: дифференцируема везде кроме начала координат:
  
  $\frac{d}{dx}\text{atan2}(y,x) = \frac{-y}{x^2+y^2}$

- **Трилинейная интерполяция**: линейная операция по весам

### **2.4 Построение гистограмм градиентов**

**Операции над изображением:**
1. **Вращение окрестности** - координатная система выравнивается по ориентации ключевой точки
2. **Разбиение на 4×4 субрегиона** - окно 16×16 пикселей
3. **Вычисление 8-биновых гистограмм** для каждого субрегиона с трилинейной интерполяцией:
   - Интерполяция по пространственным координатам (x, y)
   - Интерполяция по ориентации (θ)

**Вычисление производных (backpropagation):**
- **Вращение координат**: линейное преобразование, якобиан - матрица вращения
- **Билинейная интерполяция**: дифференцируема, градиенты по ближайшим пикселям
- **Накопление в гистограммах**: суммирование взвешенных вкладов, производные по весам интерполяции

### **2.5 Нормализация дескрипторов**

**Операции над изображением:**
1. **L2-нормализация**:
   
   $\mathbf{d}_{\text{norm}} = \frac{\mathbf{d}}{||\mathbf{d}||_2}$

2. **Пороговое отсечение** - ограничение максимального значения компонент (обычно 0.2)
3. **Повторная нормализация** для устойчивости к изменениям освещенности

**Вычисление производных (backpropagation):**
- **L2-нормализация**: дифференцируемая операция:
  
  $\frac{\partial}{\partial d_i}\left(\frac{d_j}{||\mathbf{d}||}\right) = \frac{\delta_{ij}||\mathbf{d}|| - d_j\frac{d_i}{||\mathbf{d}||}}{||\mathbf{d}||^2}$

- **Пороговое отсечение**: недифференцируемо в точке отсечения, можно заменить на мягкую версию:
  
  $\text{soft\_clip}(x, threshold) = \frac{threshold \cdot \tanh(x/threshold) + threshold}{2}$
## **3. Реализация SIFT (псевдокод)**

### **3.1 Алгоритм в псевдокоде**

**Вход:** изображение $I(x,y)$  
**Выход:** ключевые точки $K$ с дескрипторами $D$

---

**Шаг 1: Построение гауссовой пирамиды**

Для октав $o = 0,1,\dots,O-1$ и уровней $s = 0,1,\dots,S+2$:

$$
L(x,y,\sigma) = G(x,y,\sigma) \ast I(x,y)
$$

где $G(x,y,\sigma) = \frac{1}{2\pi\sigma^2}\exp\left(-\frac{x^2+y^2}{2\sigma^2}\right)$,  
$\sigma = \sigma_0 \cdot 2^{o+s/S}$, $\sigma_0 = 1.6$.

---

**Шаг 2: Вычисление разности гауссианов (DoG)**

Для каждой октавы:

$$
D(x,y,\sigma) = L(x,y,k\sigma) - L(x,y,\sigma)
$$

где $k = 2^{1/S}$.

---

**Шаг 3: Поиск локальных экстремумов**

Для каждого пикселя $(x,y,\sigma)$ в $D$:

Проверить, является ли $D(x,y,\sigma)$ локальным экстремумом в $3\times3\times3$ окне (8 соседей в том же масштабе + 9 в масштабе выше + 9 в масштабе ниже).

Если да → кандидат в ключевые точки.

---

**Шаг 4: Уточнение координат ключевой точки**

Квадратичная аппроксимация:

$$
\Delta\mathbf{x} = -\mathbf{H}^{-1} \nabla D
$$

где $\mathbf{x} = (x,y,\sigma)^T$, $\nabla D$ — градиент $D$,  
$\mathbf{H}$ — матрица Гессе (гессиан) размерности $3\times3$:

$$
\mathbf{H} = \begin{bmatrix}
\frac{\partial^2 D}{\partial x^2} & \frac{\partial^2 D}{\partial x \partial y} & \frac{\partial^2 D}{\partial x \partial \sigma} \\
\frac{\partial^2 D}{\partial y \partial x} & \frac{\partial^2 D}{\partial y^2} & \frac{\partial^2 D}{\partial y \partial \sigma} \\
\frac{\partial^2 D}{\partial \sigma \partial x} & \frac{\partial^2 D}{\partial \sigma \partial y} & \frac{\partial^2 D}{\partial \sigma^2}
\end{bmatrix}
$$

Если $|\Delta\mathbf{x}| > 0.5$ в любом измерении → сдвинуть точку и повторить.

---

**Шаг 5: Фильтрация слабых точек**

1. **По контрасту:** если $|D(\mathbf{x})| < T_{\text{contrast}}$ → отбросить.
   Обычно $T_{\text{contrast}} = 0.03$.

2. **По краям:** используя матрицу Гессе $H_{2\times2}$ для пространственных координат:

   $$
   \text{Tr}(\mathbf{H})^2 / \text{Det}(\mathbf{H}) < \frac{(r+1)^2}{r}
   $$
   
   где $r = 10$ (пороговое отношение собственных значений).
   Если условие не выполняется → точка на краю, отбросить.

---

**Шаг 6: Назначение ориентации**

Для каждой ключевой точки в масштабе $\sigma$:

1. Вычислить градиенты в окрестности радиуса $3\sigma$:

   $$
   m(x,y) = \sqrt{(L(x+1,y)-L(x-1,y))^2 + (L(x,y+1)-L(x,y-1))^2}
   $$
   
   $$
   \theta(x,y) = \arctan_2(L(x,y+1)-L(x,y-1),\ L(x+1,y)-L(x-1,y))
   $$

2. Построить гистограмму из 36 бинов (по 10°):

   $$
   h(\theta_k) = \sum_{x,y} w(x,y) \cdot m(x,y) \cdot \delta(\theta(x,y) \in \text{бин}_k)
   $$
   
   где $w(x,y) = \exp\left(-\frac{(x-x_0)^2+(y-y_0)^2}{2(1.5\sigma)^2}\right)$ — гауссов вес.

3. Найти пики гистограммы: ориентации с $h > 0.8 \cdot h_{\max}$.

---

**Шаг 7: Построение дескрипторов**

Для каждой ориентированной ключевой точки:

1. Взять окно $16\times16$ пикселей (в масштабе ключевой точки).
2. Разделить на $4\times4$ клетки (по $4\times4$ пикселя каждая).
3. Для каждой клетки вычислить 8-бинную гистограмму ориентаций (трилинейная интерполяция):

   - Интерполяция по пространству ($x$, $y$)
   - Интерполяция по ориентации ($\theta$)
   
   Вклад каждого градиента распределяется между соседними бинами и клетками.

4. Получить вектор из $4\times4\times8 = 128$ элементов.

---

**Шаг 8: Нормализация дескрипторов**

1. **L2-нормализация:**
   
   $$
   \mathbf{v} = \frac{\mathbf{d}}{\|\mathbf{d}\|_2}
   $$

2. **Пороговое отсечение (clipping):**
   
   $$
   v_i' = \min(v_i, 0.2)
   $$

3. **Повторная нормализация:**
   
   $$
   \mathbf{d}_{\text{final}} = \frac{\mathbf{v}'}{\|\mathbf{v}'\|_2}
   $$



**4. Сверточная нейронная сеть: ResNet и затухающие градиенты**

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split


class SyntheticImageDataset(Dataset):
    """Синтетический датасет с изображениями разных паттернов.
    
    Создаёт изображения 5 классов с характерными паттернами:
    - Класс 0: Вертикальные красные полосы
    - Класс 1: Горизонтальные зелёные полосы
    - Класс 2: Синие круги
    - Класс 3: Жёлтые квадраты
    - Класс 4: Диагональные фиолетовые полосы
    
    Args:
        num_samples: Количество изображений
        num_classes: Количество классов
        image_size: Размер изображения (квадратное)
    """
    
    def __init__(self, num_samples: int = 1000, num_classes: int = 5, image_size: int = 64):
        self.num_samples = num_samples
        self.num_classes = num_classes
        self.image_size = image_size
        self.data = []
        self.labels = []
        
        self._generate_data()
    
    def _generate_data(self) -> None:
        """Генерирует синтетические изображения."""
        for i in range(self.num_samples):
            img = torch.rand(3, self.image_size, self.image_size)
            label = i % self.num_classes
            
            if label == 0:  # Вертикальные красные полосы
                for x in range(self.image_size):
                    if x % 10 < 5:
                        img[0, :, x] = 0.8
                        img[1, :, x] = 0.2
                        img[2, :, x] = 0.2
                        
            elif label == 1:  # Горизонтальные зелёные полосы
                for y in range(self.image_size):
                    if y % 10 < 5:
                        img[0, y, :] = 0.2
                        img[1, y, :] = 0.8
                        img[2, y, :] = 0.2
                        
            elif label == 2:  # Синие круги
                cx = np.random.randint(20, self.image_size - 20)
                cy = np.random.randint(20, self.image_size - 20)
                r = np.random.randint(10, 20)
                for x in range(self.image_size):
                    for y in range(self.image_size):
                        if (x - cx)**2 + (y - cy)**2 <= r**2:
                            img[0, x, y] = 0.2
                            img[1, x, y] = 0.2
                            img[2, x, y] = 0.8
                            
            elif label == 3:  # Жёлтые квадраты
                x0 = np.random.randint(10, self.image_size - 30)
                y0 = np.random.randint(10, self.image_size - 30)
                size = np.random.randint(15, 25)
                img[0, x0:x0+size, y0:y0+size] = 0.8
                img[1, x0:x0+size, y0:y0+size] = 0.8
                img[2, x0:x0+size, y0:y0+size] = 0.2
                
            elif label == 4:  # Диагональные фиолетовые полосы
                for x in range(self.image_size):
                    for y in range(self.image_size):
                        if (x + y) % 20 < 10:
                            img[0, x, y] = 0.8
                            img[1, x, y] = 0.2
                            img[2, x, y] = 0.8

            self.data.append(img)
            self.labels.append(label)

    def __len__(self) -> int:
        return self.num_samples

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        return self.data[idx], self.labels[idx]


# Создание датасета
NUM_SAMPLES = 500
NUM_CLASSES = 5
IMAGE_SIZE = 64
BATCH_SIZE = 32
TRAIN_RATIO = 0.8

dataset = SyntheticImageDataset(
    num_samples=NUM_SAMPLES,
    num_classes=NUM_CLASSES,
    image_size=IMAGE_SIZE
)

train_size = int(TRAIN_RATIO * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Размер обучающей выборки: {len(train_dataset)}")
print(f"Размер валидационной выборки: {len(val_dataset)}")

In [ ]:
class SimpleCNN(nn.Module):
    """Простая CNN для классификации изображений.
    
    Архитектура: Conv -> Pool -> Conv -> Pool -> FC
    
    Args:
        num_classes: Количество классов для классификации
    """
    
    def __init__(self, num_classes: int = 5):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(32 * 16 * 16, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))  # 64x64 -> 32x32
        x = self.pool(F.relu(self.conv2(x)))  # 32x32 -> 16x16
        x = x.view(x.size(0), -1)
        return self.fc(x)


# Инициализация моделей
cnn_model = SimpleCNN(num_classes=NUM_CLASSES)

resnet_model = resnet18(weights=None)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, NUM_CLASSES)

print("Модели инициализированы:")
print(f"  SimpleCNN: {sum(p.numel() for p in cnn_model.parameters())} параметров")
print(f"  ResNet18: {sum(p.numel() for p in resnet_model.parameters())} параметров")

In [ ]:
def compute_gradient_norms(model: nn.Module) -> List[float]:
    """Вычисляет нормы градиентов для всех параметров модели.
    
    Args:
        model: PyTorch модель
        
    Returns:
        Список норм градиентов
    """
    norms = []
    for param in model.parameters():
        if param.grad is not None:
            norms.append(param.grad.norm().item())
    return norms


def train_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer
) -> Tuple[float, float, float]:
    """Обучает модель одну эпоху.
    
    Args:
        model: Модель для обучения
        loader: DataLoader с данными
        criterion: Функция потерь
        optimizer: Оптимизатор
        
    Returns:
        Кортеж (средний loss, accuracy, средняя норма градиентов)
    """
    model.train()
    total_loss = 0
    correct = 0
    epoch_gradients = []

    for x, y in loader:
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()

        gradients = compute_gradient_norms(model)
        if gradients:
            epoch_gradients.append(np.mean(gradients))

        optimizer.step()

        total_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct / len(loader.dataset)
    avg_gradient = np.mean(epoch_gradients) if epoch_gradients else 0

    return avg_loss, accuracy, avg_gradient


def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    epochs: int = 5
) -> Tuple[List[float], List[float], List[float]]:
    """Обучает модель заданное количество эпох.
    
    Args:
        model: Модель для обучения
        train_loader: DataLoader с обучающими данными
        criterion: Функция потерь
        optimizer: Оптимизатор
        epochs: Количество эпох
        
    Returns:
        Кортеж списков (losses, accuracies, gradients)
    """
    losses, accuracies, gradients = [], [], []

    for epoch in range(epochs):
        loss, acc, grad = train_epoch(model, train_loader, criterion, optimizer)
        losses.append(loss)
        accuracies.append(acc)
        gradients.append(grad)
        print(f"Эпоха {epoch+1}: loss={loss:.4f}, accuracy={acc:.4f}")

    return losses, accuracies, gradients

In [ ]:
# Обучение моделей
LEARNING_RATE = 0.001
NUM_EPOCHS = 5

criterion = nn.CrossEntropyLoss()

# Обучение CNN
print("Обучение SimpleCNN:")
print("-" * 40)
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=LEARNING_RATE)
cnn_losses, cnn_accuracies, cnn_gradients = train_model(
    cnn_model, train_loader, criterion, cnn_optimizer, epochs=NUM_EPOCHS
)

# Обучение ResNet
print("\nОбучение ResNet18:")
print("-" * 40)
resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=LEARNING_RATE)
resnet_losses, resnet_accuracies, resnet_gradients = train_model(
    resnet_model, train_loader, criterion, resnet_optimizer, epochs=NUM_EPOCHS
)

In [ ]:
def get_layer_gradients(model: nn.Module, layer_names: List[str]) -> dict:
    """Извлекает градиенты по указанным слоям.
    
    Args:
        model: Модель
        layer_names: Список имён слоёв для извлечения
        
    Returns:
        Словарь {имя_слоя: норма_градиента}
    """
    gradients = {}
    for name, param in model.named_parameters():
        if 'weight' in name and param.grad is not None:
            for layer_name in layer_names:
                if layer_name in name:
                    gradients[layer_name] = param.grad.norm().item()
                    break
    return gradients


# Визуализация результатов
fig = plt.figure(figsize=(15, 10))

# 1. График потерь
ax1 = fig.add_subplot(2, 3, 1)
ax1.plot(cnn_losses, 'b-o', label='SimpleCNN')
ax1.plot(resnet_losses, 'r-s', label='ResNet18')
ax1.set_xlabel('Эпоха')
ax1.set_ylabel('Loss')
ax1.set_title('Функция потерь')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. График точности
ax2 = fig.add_subplot(2, 3, 2)
ax2.plot(cnn_accuracies, 'b-o', label='SimpleCNN')
ax2.plot(resnet_accuracies, 'r-s', label='ResNet18')
ax2.set_xlabel('Эпоха')
ax2.set_ylabel('Accuracy')
ax2.set_title('Точность на обучении')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. График градиентов
ax3 = fig.add_subplot(2, 3, 3)
ax3.plot(cnn_gradients, 'b-o', label='SimpleCNN')
ax3.plot(resnet_gradients, 'r-s', label='ResNet18')
ax3.set_xlabel('Эпоха')
ax3.set_ylabel('Норма градиентов')
ax3.set_title('Средняя норма градиентов')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Градиенты по слоям CNN
ax4 = fig.add_subplot(2, 3, 4)
cnn_layer_grads = get_layer_gradients(cnn_model, ['conv1', 'conv2', 'fc'])
if cnn_layer_grads:
    layers = list(cnn_layer_grads.keys())
    grads = list(cnn_layer_grads.values())
    ax4.bar(range(len(layers)), grads, color='steelblue')
    ax4.set_xticks(range(len(layers)))
    ax4.set_xticklabels(layers, rotation=45)
ax4.set_ylabel('Норма градиентов')
ax4.set_title('Градиенты по слоям SimpleCNN')

# 5. Градиенты по слоям ResNet
ax5 = fig.add_subplot(2, 3, 5)
resnet_layers = ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']
resnet_layer_grads = get_layer_gradients(resnet_model, resnet_layers)
if resnet_layer_grads:
    grads = [resnet_layer_grads.get(layer, 0) for layer in resnet_layers]
    ax5.bar(range(len(resnet_layers)), grads, color='indianred')
    ax5.set_xticks(range(len(resnet_layers)))
    ax5.set_xticklabels(resnet_layers, rotation=45)
ax5.set_ylabel('Норма градиентов')
ax5.set_title('Градиенты по слоям ResNet18')

# 6. Сравнение финальных градиентов
ax6 = fig.add_subplot(2, 3, 6)
comparison = [cnn_gradients[-1], resnet_gradients[-1]]
ax6.bar(['SimpleCNN', 'ResNet18'], comparison, color=['steelblue', 'indianred'])
ax6.set_ylabel('Норма градиентов (эпоха 5)')
ax6.set_title('Сравнение градиентов')

plt.tight_layout()
plt.show()

In [ ]:
# Анализ результатов
print("=" * 60)
print("АНАЛИЗ РЕЗУЛЬТАТОВ")
print("=" * 60)

print(f"\n1. Финальные показатели:")
print(f"   SimpleCNN: Loss={cnn_losses[-1]:.4f}, Accuracy={cnn_accuracies[-1]:.4f}")
print(f"   ResNet18:  Loss={resnet_losses[-1]:.4f}, Accuracy={resnet_accuracies[-1]:.4f}")

print(f"\n2. Изменение градиентов за обучение:")
cnn_grad_change = (cnn_gradients[-1] - cnn_gradients[0]) / cnn_gradients[0] * 100
resnet_grad_change = (resnet_gradients[-1] - resnet_gradients[0]) / resnet_gradients[0] * 100
print(f"   SimpleCNN: {cnn_grad_change:+.1f}%")
print(f"   ResNet18:  {resnet_grad_change:+.1f}%")

print(f"\n3. Отношение градиентов между слоями:")
cnn_layer_grads = get_layer_gradients(cnn_model, ['conv1', 'conv2', 'fc'])
resnet_layer_grads = get_layer_gradients(resnet_model, ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc'])

if 'conv1' in cnn_layer_grads and 'conv2' in cnn_layer_grads:
    ratio = cnn_layer_grads['conv2'] / cnn_layer_grads['conv1']
    print(f"   SimpleCNN (conv2/conv1): {ratio:.3f}")

if 'layer1' in resnet_layer_grads and 'layer4' in resnet_layer_grads:
    ratio = resnet_layer_grads['layer4'] / resnet_layer_grads['layer1']
    print(f"   ResNet18 (layer4/layer1): {ratio:.3f}")

print(f"\n4. Выводы:")
print("   - ResNet показывает более стабильные градиенты благодаря skip-connections")
print("   - В SimpleCNN градиенты затухают сильнее в ранних слоях")
print("   - Skip-connections в ResNet предотвращают проблему затухающих градиентов")